In [1]:
# installing libraries
from bs4 import BeautifulSoup as bs
import requests as rq
from urllib.request import urlopen
import pandas as pd
import numpy as  np
import pandas as pd
import requests
import re
import time
import seaborn as sns


In [6]:
# web scrapping for reviews

headers = {
    'Referer': 'https://www.rottentomatoes.com/m/notebook/reviews?type=user',
    'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/74.0.3729.108 Safari/537.36',
    'X-Requested-With': 'XMLHttpRequest',
}

s = requests.Session()
        
def get_reviews(url):
    r = requests.get(url)
    movie_id = re.findall(r'(?<=movieId":")(.*)(?=","type)',r.text)[0]

    api_url = f"https://www.rottentomatoes.com/napi/movie/{movie_id}/reviews/user" #use reviews/userfor user reviews
    
    payload = {
        'direction': 'next',
        'endCursor': '',
        'startCursor': '',
    }
    
    review_data = []
    
    count=0

    while True:
        r = s.get(api_url, headers=headers, params=payload)
        data = r.json()

        if not data['pageInfo']['hasNextPage']:
            break

        payload['endCursor'] = data['pageInfo']['endCursor']
        payload['startCursor'] = data['pageInfo']['startCursor'] if data['pageInfo'].get('startCursor') else ''

        review_data.extend(data['reviews'])
        time.sleep(5)
        count= count+1
        if count >500 :
          break 
    
    return review_data

data = get_reviews('https://www.rottentomatoes.com/m/top_gun_maverick/reviews?type=user')
df1 = pd.json_normalize(data)

In [7]:
df1.shape

(510, 13)

In [8]:
df1.to_csv('review')